## Getting data and install packages

In [1]:
!pip install -Uqq datasets pythainlp==2.2.4 transformers==4.4.0 tensorflow==2.4.0 tensorflow_text emoji seqeval sentencepiece fuzzywuzzy
!npx degit --force https://github.com/vistec-AI/thai2transformers#dev

     |████████████████████████████████| 225kB 28.5MB/s 
     |████████████████████████████████| 8.9MB 27.3MB/s 
     |████████████████████████████████| 2.1MB 30.6MB/s 
     |████████████████████████████████| 394.7MB 38kB/s 
     |████████████████████████████████| 4.3MB 43.1MB/s 
     |████████████████████████████████| 133kB 56.7MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 1.2MB 47.3MB/s 
     |████████████████████████████████| 112kB 58.7MB/s 
     |████████████████████████████████| 245kB 57.4MB/s 
     |████████████████████████████████| 747kB 49.1MB/s 
     |████████████████████████████████| 901kB 47.4MB/s 
     |████████████████████████████████| 3.3MB 47.8MB/s 
ERROR: tensorflow-text 2.5.0 has requirement tensorflow<2.6,>=2.5.0, but you'll have tensorflow 2.4.0 which is incompatible.
npx: installed 1 in 2.231s
> destination directory is not empty. Using --force, continuing
> cloned vistec-AI/thai2transformers#dev


In [2]:
%load_ext autoreload
%autoreload 2

import pythainlp, transformers
pythainlp.__version__, transformers.__version__ #fix pythainlp to stabilize word tokenization for metrics

('2.2.4', '4.4.0')

In [3]:
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)

#transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

#thai2transformers
import thai2transformers
from thai2transformers.metrics import (
    squad_newmm_metric,
    question_answering_metrics,
)
from thai2transformers.preprocess import (
    prepare_qa_train_features
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

from tqdm import tqdm

In [4]:
from thai2transformers.tokenizers import ThaiWordsNewmmTokenizer
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')

In [5]:
!gdown --id 1-8IU8qyry-yPXwQ7AXz0GHIgn19QKGZP
!gdown --id 1-J0eqf4ig7cP8bMPRgSFUejshnBFTZoq
!gdown --id 1-IIJFl4AGNr7rRax4YSQTTm7j12YJ0ya

Downloading...
From: https://drive.google.com/uc?id=1-8IU8qyry-yPXwQ7AXz0GHIgn19QKGZP
To: /content/thaisum.csv
2.94GB [00:54, 53.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-J0eqf4ig7cP8bMPRgSFUejshnBFTZoq
To: /content/validation_set.csv
118MB [00:01, 93.3MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1-IIJFl4AGNr7rRax4YSQTTm7j12YJ0ya
To: /content/test_set.csv
119MB [00:02, 48.3MB/s]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/thaisum.csv')
val_df = pd.read_csv('/content/drive/MyDrive/validation_set.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test_set.csv')
df = pd.concat([df, val_df, test_df], axis=0)

In [8]:
data_test = df['body'][369868:380868].tolist()
data_test[:3]

['พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ครม.ตั้ง เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้ แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธุ์ ประธาน กรธ. ยังคงยืนยันเดินหน้าต่อไป แม้การเลือกตั้งตามรูปแบบใหม

In [9]:
from tqdm import tqdm
from pythainlp.tokenize import word_tokenize, sent_tokenize

def segment_sent(data):
  ret = []
  for text in tqdm(data):
    ret.append(sent_tokenize(text))
  return ret

In [10]:
data_segmented_sentence = segment_sent(data_test)

100%|██████████| 11000/11000 [08:53<00:00, 20.61it/s]


In [11]:
data_segmented_sentence[:3]

[['พอไม่ได้ดังใจก็ร่างใหม่ ',
  'แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ',
  'ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน ',
  'สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม ',
  'แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ',
  'ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ',
  'ครม.ตั้ง ',
  'เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ ',
  'อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ',
  'ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้ ',
  'แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. ',
  'สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธ

In [12]:
def gold_summary(df, num_train, num_val, num_test):
  return df.iloc[num_train+num_val:num_train+num_val+num_test,:]['summary'].tolist()

In [13]:
gold_summaries = gold_summary(df, 358868, 11000, 11000)
gold_summaries_sentence_segmented = segment_sent(gold_summaries)

100%|██████████| 11000/11000 [00:26<00:00, 419.75it/s]


## Metrics

In [14]:
def evaluate_rouge(cands, refs, tokenizer):
  cands_tokenized = tokenize_with_space(cands, tokenizer)
  refs_tokenized = tokenize_with_space(refs, tokenizer)
  r1, r2, rl = cal_rouge_score(refs_tokenized, cands_tokenized)
  # BERTScore = cal_batch_bert_score(cands, refs)
  return r1, r2, rl

def tokenize_with_space(texts, tokenizer):
  output = []
  encoded_texts = tokenizer(texts, max_length=416, truncation=True)
  for text in encoded_texts['input_ids']:
    tokenized_text = " ".join(tokenizer.convert_ids_to_tokens(text, skip_special_tokens=True))
    if(len(tokenized_text)==0):
      output.append("")
      continue
    if(tokenized_text[0]=="▁"): 
      tokenized_text = tokenized_text[1:]
    output.append(tokenized_text.strip())
  return output

In [15]:
!pip install rouge

In [16]:
from rouge import Rouge 
rouge = Rouge()
def cal_rouge_score(hyps, refs, get_average_f1=True):
  '''
  argument: cands, refs [list of string], get_average_f1=True
  return dict of r1, r2, rl score
  if get_average_f1 == True return mean of rouge-1, rouge-2, rouge-L
  '''
  r1 = dict(); r1['precision'] = []; r1['recall'] = []; r1['f1'] = []
  r2 = dict(); r2['precision'] = []; r2['recall'] = []; r2['f1'] = []
  rl = dict(); rl['precision'] = []; rl['recall'] = []; rl['f1'] = []
  for hyp, ref in zip(hyps, refs):
    score = {}
    if(len(hyp)==0 or len(ref)==0):
      score = {
          'rouge-1': {
              'p': 0,
              'r': 0,
              'f': 0
          },
          'rouge-2': {
              'p': 0,
              'r': 0,
              'f': 0
          },
          'rouge-l': {
              'p': 0,
              'r': 0,
              'f': 0
          }
      }
    else: score = rouge.get_scores(hyp, ref)[0]
    r1['precision'].append(score['rouge-1']['p'])
    r1['recall'].append(score['rouge-1']['r'])
    r1['f1'].append(score['rouge-1']['f'])
    
    r2['precision'].append(score['rouge-2']['f'])
    r2['recall'].append(score['rouge-2']['f'])
    r2['f1'].append(score['rouge-2']['f'])

    rl['precision'].append(score['rouge-l']['f'])
    rl['recall'].append(score['rouge-l']['f'])
    rl['f1'].append(score['rouge-l']['f'])
  # score = rouge.get_scores(cands, refs, avg=True)
  # r1['precision_average'] = score['rouge-1']['p']
  # r1['recall_average'] = score['rouge-1']['r']
  # r1['f1_average'] = score['rouge-1']['f']
  
  # r2['precision_average'] = score['rouge-2']['f']
  # r2['recall_average'] = score['rouge-2']['f']
  # r2['f1_average'] = score['rouge-2']['f']

  # rl['precision_average'] = score['rouge-l']['f']
  # rl['recall_average'] = score['rouge-l']['f']
  # rl['f1_average'] = score['rouge-l']['f']
  if(get_average_f1==True): return sum(r1['f1'])/len(r1['f1']), sum(r2['f1'])/len(r2['f1']), sum(rl['f1'])/len(rl['f1'])
  else: return r1, r2, rl

In [17]:
cands = ['test test test test test test bad']
refs = ['test test']

r1, r2, rl = cal_rouge_score(cands, refs)
print(r1)
print(r2)
print(rl)

0.4444444409876544
0.2857142832653061
0.6666666622222223


In [18]:
!pip install bert_score==0.3.7
from bert_score import score
import numpy as np

     |████████████████████████████████| 61kB 6.6MB/s 


In [19]:
import gc

def cal_bert_score(cands, refs, get_average_f1=True):
  '''
  arguments: cands, refs
  return array of presicion, recall, f1, presicion_average, recall_average, f1_average
  if get_average == True return mean of BERTScore
  '''
  p, r, f1 = score(cands, refs, lang="others", verbose=False)
  p = p.numpy()
  r = r.numpy()
  f1 = f1.numpy()
  if(get_average_f1==True): return f1.mean()
  else: return p, r, f1

def cal_batch_bert_score(cands, refs, get_average_f1=True, batch_size=8):
  f1_average = []
  for i in tqdm(range(0,len(cands),batch_size)):
    cand_batch = cands[i:i+batch_size]
    ref_batch = refs[i:i+batch_size]
    res = cal_bert_score(cand_batch, ref_batch)
    f1_average.append(res)
    gc.collect()
  print(f1_average)
  return sum(f1_average)/len(f1_average)

In [20]:
%%time
refs = ['เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่งโรดแม็ปมองได้สองแบบ คือ มีลำดับขั้นตอนและการกำหนดช่วงเวลา โดยเริ่มต้นจากการประกาศใช้รัฐธรรมนูญ แต่ขณะนี้รัฐธรรมนูญยังไม่ประกาศใช้ จึงยังเริ่มนับหนึ่งไม่ถูก จากนั้นเข้าสู่ขั้นตอนการร่างกฎหมายประกอบร่างรัฐธรรมนูญหรือกฎหมายลูก ภายใน 240 วัน ก่อนจะส่งกลับให้ สนช.พิจารณา ภายใน 2 เดือน\xa0,นายวิษณุ กล่าวต่อว่า หากมีการแก้ไขก็จะมีการพิจารณาร่วมกับ กรธ.อีก 1 เดือน ก่อนนำขึ้นทูลเกล้าฯ ทรงลงพระปรมาภิไธย ภายใน 90 วัน และจะเข้าสู่การเลือกตั้งภายในระยะเวลา 5 เดือน ซึ่งทั้งหมดนี้คือโรดแม็ปที่ยังเป็นแบบเดิมอยู่ ส่วนเดิมที่กำหนดวันเลือกตั้งไว้ภายในปี 60 นั้น เพราะมาจากสมมติฐานของขั้นตอนเดิมทั้งหมด แต่เมื่อมีเหตุสวรรคตทุกอย่างจึงต้องเลื่อนออกไป ส่วนการพิจารณากฎหมายลูกทั้งหมด 4 ฉบับ ขณะนี้กรธ.พิจารณาแล้วเสร็จ 2 ฉบับ คือ พ.ร.ป.พรรคการเมือง และพ.ร.ป. คณะกรรมการการเลือกตั้ง แต่ พ.ร.ป.การเลือกตั้งควรจะพิจารณาได้เร็วกลับล่าช้า ดังนั้น กรธ.จะต้องออกชี้แจงถึงเหตุผลว่าทำไมพิจารณากฎหมายดังกล่าวล่าช้ากว่ากำหนด ส่งผลให้เกิดข้อสงสัยจนถึงทุกวันนี้ ส่วนกรณีที่ สนช. ระบุว่า มีกฎหมายเข้าสู่การพิจารณาของ สนช.เป็นจำนวนมาก ทำให้ส่งผลกระทบต่อโรดแม็ปนั้น รัฐบาลเคยบอกไว้แล้วว่าในช่วงนี้ของโรดแม็ปกฎหมายจะเยอะกว่าที่ผ่านมา ดังนั้น สนช.จะต้องบริหารจัดการกันเอง เพราะได้มีการเพิ่มสมาชิก สนช.ให้แล้ว.']
cands = ['เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.)']
f1_average = cal_bert_score(cands, refs)


CPU times: user 18.4 s, sys: 4.82 s, total: 23.2 s
Wall time: 31.4 s


## Oracle A

In [21]:
import collections as coll
# stopwords = pkgutil.get_data(__package__, 'smart_common_words.txt')
# stopwords = stopwords.decode('ascii').split('\n')
# stopwords = {key.strip(): 1 for key in stopwords}

def _get_ngrams_count(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    ngram_dic = coll.defaultdict(int)
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_dic[tuple(text[i:i + n])] += 1
    return ngram_dic

def _get_ngrams(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

def _get_word_ngrams_list(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    text = sum(text, [])
    ngram_set = []
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.append(tuple(text[i:i + n]))
    return ngram_set

def _get_word_ngrams(n, sentences, do_count=False):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    # words = _split_into_words(sentences)

    words = sum(sentences, [])
    # words = [w for w in words if w not in stopwords]
    if do_count:
        return _get_ngrams_count(n, words)
    return _get_ngrams(n, words)
  
def cal_rouge(evaluated_ngrams, reference_ngrams):
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))
    return {"f": f1_score, "p": precision, "r": recall}

def greedy_selection(doc_sent_list, abstract_sent_list, summary_size):
  max_rouge = 0.0
  abstract = sum(abstract_sent_list, [])
  abstract = ' '.join(abstract).split()
  sents = [' '.join(s).split() for s in doc_sent_list]
  evaluated_1grams = [_get_word_ngrams(1, [sent]) for sent in sents]
  reference_1grams = _get_word_ngrams(1, [abstract])
  evaluated_2grams = [_get_word_ngrams(2, [sent]) for sent in sents]
  reference_2grams = _get_word_ngrams(2, [abstract])

  selected = []
  for s in range(summary_size):
      cur_max_rouge = max_rouge
      cur_id = -1
      for i in range(len(sents)):
          if (i in selected):
              continue
          c = selected + [i]
          candidates_1 = [evaluated_1grams[idx] for idx in c]
          candidates_1 = set.union(*map(set, candidates_1))
          candidates_2 = [evaluated_2grams[idx] for idx in c]
          candidates_2 = set.union(*map(set, candidates_2))
          rouge_1 = cal_rouge(candidates_1, reference_1grams)['f']
          rouge_2 = cal_rouge(candidates_2, reference_2grams)['f']
          rouge_score = rouge_1 + rouge_2
          if rouge_score > cur_max_rouge:
              cur_max_rouge = rouge_score
              cur_id = i
      if (cur_id == -1):
          return selected
      selected.append(cur_id)
      max_rouge = cur_max_rouge

  return sorted(selected)

In [22]:
def oracle_a(paragraph_raw, gold_summaries):
  ret = []
  sent_selected = []
  for docs, abs in tqdm(zip(paragraph_raw, gold_summaries)):
    t_docs = [[x] for x in docs]
    t_abs = [[x] for x in abs]
    selected = greedy_selection(t_docs, t_abs, 4)
    new_docs = []
    sent_selected.append(selected)
    for idx in selected:
      new_docs.append(docs[idx])
    ret.append("".join(new_docs))
  return ret, sent_selected

In [23]:
data_segmented_sentence[0]

['พอไม่ได้ดังใจก็ร่างใหม่ ',
 'แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ',
 'ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน ',
 'สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม ',
 'แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ',
 'ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ',
 'ครม.ตั้ง ',
 'เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ ',
 'อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ',
 'ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้ ',
 'แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. ',
 'สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธุ์ ',
 'ประธ

In [24]:
summaries_oracle_a = oracle_a(data_segmented_sentence, gold_summaries_sentence_segmented)

11000it [00:13, 807.48it/s]


In [25]:
# summaries_oracle_a[0][:10]
text_summaries_oracle_a = summaries_oracle_a[0]
text_summaries_oracle_a[:5]

['ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ',
 'เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง กีเก ซานเชซ ฟลอเรส\xa0 กุนซือเลือดกระทิงของทีมวัตฟอร์ด\xa0 ',
 'เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภคพังพินาศ ทางการเร่งระดมเจ้าหน้าที่ตำรวจ เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน ',
 'ระหว่าง 17-18 พ.ค. ที่ห้องอาซัวร์ เรเนซองส์ เป็นศูนย์รวมของความคิดสร้างสรรค์และคอมมูนิตี้คนรุ่นใหม่ ล่าสุด อุสรา ยงปิยะกุล คว้าศิลปินไทยรุ่นใหม่จากนิวยอร์ก กันตภณ เมธีกุล มาร่วมคอลลาโบเรชั่น สินค้าลิมิเต็ด อิดิชั่น ODS X GONGKAN ให้คนรักงานศิลป์คนไทย ริเริ่มจัดบุฟเฟต์ทุเรียนจนเป็นเทรนด์นิยม ปีนี้ ',
 'บ้านตะวันลับ Twilight Villa ถูกจัดตั้งขึ้นในปี

In [26]:
gold_summaries[0]

'มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง'

In [27]:
r1, r2, rl = evaluate_rouge(text_summaries_oracle_a, gold_summaries, tokenizer)
print(r1, r2, rl)

0.7808340934426083 0.7448053091355908 0.76082355242242


In [28]:
BERTScore = cal_batch_bert_score(text_summaries_oracle_a, gold_summaries, batch_size=128)

100%|██████████| 86/86 [25:43<00:00, 17.94s/it]

[0.89319736, 0.8836837, 0.88032484, 0.8779398, 0.8858798, 0.8809267, 0.88100886, 0.87674797, 0.89173806, 0.87507653, 0.8732444, 0.88800573, 0.8811226, 0.8898343, 0.8793214, 0.8658464, 0.87618726, 0.8603794, 0.88456786, 0.8859944, 0.87576336, 0.8901665, 0.89399076, 0.87415874, 0.87273276, 0.880453, 0.8796413, 0.8908386, 0.8764343, 0.87796783, 0.88312936, 0.8836874, 0.8804668, 0.8573898, 0.87110853, 0.879704, 0.86816835, 0.890043, 0.8747535, 0.890775, 0.88171005, 0.87800777, 0.89452434, 0.8808714, 0.88298535, 0.8807734, 0.88243896, 0.8797245, 0.88437307, 0.88141817, 0.8905537, 0.88761854, 0.8906372, 0.88758254, 0.87844, 0.87384605, 0.8899583, 0.8804733, 0.8992124, 0.8841306, 0.890375, 0.87512314, 0.87130505, 0.88755757, 0.8972032, 0.8746815, 0.8713199, 0.8795327, 0.8980079, 0.8765938, 0.8822874, 0.8898929, 0.89170563, 0.8693235, 0.8815782, 0.8730935, 0.8821719, 0.8828261, 0.8888686, 0.8984972, 0.89771914, 0.89279693, 0.8796184, 0.87243587, 0.8880561, 0.8856912]


In [29]:
print(BERTScore)

0.8822086335614671


In [30]:
np.save('text_summaries_oracle_a', np.array(text_summaries_oracle_a))

## Oracle-b

In [23]:
tokenized_datasets = load_dataset('json', data_files={'train': '/content/drive/MyDrive/train.json', 'validation': '/content/drive/MyDrive/validation_true.json', 'test': '/content/drive/MyDrive/test_true.json'})

Using custom data configuration default-757701d6e8dd32b8


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-757701d6e8dd32b8/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [21]:
def get_text_from_position(dataset, tokenizer):
  start_positions = dataset['start_positions']
  end_positions = dataset['end_positions']
  texts = dataset['input_ids']
  summaries = []
  for text, i, j in zip(texts, start_positions, end_positions):
    # print(i, j)
    cut_text = text[i:j]
    decoded_text = tokenizer.decode(cut_text, skip_special_tokens=True)
    summaries.append(decoded_text)
  return summaries

In [24]:
summary_from_oracle_b = get_text_from_position(tokenized_datasets['test'], tokenizer)
summary_from_oracle_b[:3]

[',มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรค',
 'กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัตฟอร์ด เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่น',
 'ลําบาก,เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภค']

In [25]:
r1, r2, rl = evaluate_rouge(summary_from_oracle_b, gold_summaries, tokenizer)
print(r1, r2, rl)

0.8761107442453164 0.8461872369277695 0.8809886286771023


In [26]:
BERTScore = cal_batch_bert_score(summary_from_oracle_b, gold_summaries, tokenizer, batch_size=128)

100%|██████████| 86/86 [12:29<00:00,  8.71s/it]

[0.89409065, 0.8843618, 0.8899675, 0.8899961, 0.88627505, 0.89252067, 0.8815578, 0.8746747, 0.8936076, 0.8783157, 0.8806659, 0.8919265, 0.8821515, 0.8753854, 0.8856243, 0.88001186, 0.87944686, 0.89217114, 0.8903718, 0.87828004, 0.8864628, 0.88284016, 0.88387394, 0.8869653, 0.8861683, 0.8907838, 0.8867718, 0.89030886, 0.88451195, 0.8910098, 0.88960123, 0.88211375, 0.8831756, 0.8908317, 0.8877275, 0.884252, 0.88376534, 0.89573044, 0.8889476, 0.8929671, 0.8906442, 0.87785727, 0.88514423, 0.89004815, 0.8832965, 0.87747204, 0.8811549, 0.8804305, 0.8807837, 0.8852822, 0.8838256, 0.88907903, 0.8782022, 0.8890723, 0.8654926, 0.890154, 0.87815505, 0.87994677, 0.8997867, 0.8798516, 0.8912456, 0.8926846, 0.8893193, 0.8888842, 0.8862995, 0.8764006, 0.8811882, 0.8805961, 0.88683134, 0.87431276, 0.8917701, 0.8738668, 0.8968054, 0.8736851, 0.87292314, 0.8907656, 0.88553965, 0.88341355, 0.87125504, 0.89344215, 0.8814632, 0.8964179, 0.8762845, 0.8968688, 0.8857978, 0.8762522]


In [27]:
print(BERTScore)

0.8850023788075114


In [ ]:
np.save('summary_from_oracle_b', np.array(summary_from_oracle_b))

## Lead-3

In [31]:
def lead_3(data):
  ret = []
  for text in tqdm(data):
    ret.append("".join(text[:3]))
  return ret

In [32]:
summaries_lead_3 = lead_3(data_segmented_sentence)

100%|██████████| 11000/11000 [00:00<00:00, 427090.86it/s]


In [33]:
summaries_lead_3[:3]

['พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน ',
 'กีเก ซานเชซ ฟลอเรส\xa0 กุนซือเลือดกระทิงของทีมวัตฟอร์ด\xa0 เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง สำนักข่าวต่างประเทศรายงานวันที่ 27 ก.ย. ว่า กีเก ซานเชซ ฟลอเรส\xa0 ผู้จัดการทีมชาวสเปน ของ แตนอาละวาด วัตฟอร์ด\xa0 ยอมรับทีมของเขาเล่นได้ไม่ดีพอเอง ',
 'ตาย30-เจ็บ400 เส้นทางตัดขาด เข้าช่วยลำบาก เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภคพังพินาศ ทางการเร่งระดมเจ้าหน้าที่ตำรวจ ']

In [34]:
gold_summaries[:3]

['มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง',
 'กีเก ซานเชซ ฟลอเรส  กุนซือเลือดกระทิงของทีมวัตฟอร์ด  เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง',
 'เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร้างความเสียหายหลายหมู่บ้าน เบื้องต้นพบผู้เสียชีวิตแล้ว 30 ศพ บาดเจ็บ 400 ราย ระบบสาธารณูปโภคพังพินาศ']

In [35]:
r1, r2, rl = evaluate_rouge(summaries_lead_3, gold_summaries, tokenizer)
print(r1, r2, rl)

0.604528036727442 0.5596666594573724 0.6195494182077482


In [36]:
BERTScore = cal_batch_bert_score(summaries_lead_3, gold_summaries, batch_size=128)

100%|██████████| 86/86 [27:54<00:00, 19.48s/it]

[0.8345444, 0.8464756, 0.84273994, 0.8378253, 0.8387501, 0.8473702, 0.83208346, 0.8372569, 0.8413744, 0.83214194, 0.8419398, 0.84918404, 0.83472425, 0.8334775, 0.8336364, 0.832308, 0.8470652, 0.83865845, 0.8404116, 0.84029114, 0.8413036, 0.85099775, 0.8443486, 0.8459867, 0.8289015, 0.8424605, 0.83457977, 0.8530023, 0.8438095, 0.8451314, 0.83553374, 0.8405249, 0.8300036, 0.827894, 0.8329407, 0.8314443, 0.83256584, 0.83957916, 0.84691614, 0.8428999, 0.83579373, 0.8395871, 0.84907603, 0.8470104, 0.8377347, 0.8397471, 0.8291555, 0.83986306, 0.83490634, 0.8408505, 0.8430065, 0.83317757, 0.84134436, 0.83819497, 0.83277583, 0.8472707, 0.8360435, 0.83738154, 0.854439, 0.83460546, 0.8452349, 0.8466543, 0.83415437, 0.8331202, 0.8566128, 0.83887786, 0.83837813, 0.8298112, 0.84650666, 0.83458006, 0.8427532, 0.83417904, 0.8561456, 0.8271071, 0.84181404, 0.8374929, 0.8336502, 0.8468715, 0.849596, 0.8510137, 0.84463793, 0.84719455, 0.8470298, 0.83733153, 0.8330279, 0.8290791]


In [37]:
BERTScore

0.8398593144361363

In [38]:
np.save('summaries_lead_3', np.array(summaries_lead_3))